# Simple airfoil

This notebook explains how to create a simple airfoil in SHARPy.
The objective is to help the reader to create a first case in SHARPy with the minimum possible complexity.Depending on the curiosity of the reader, it can be used as a base for further discovery of the code.

## Generation of the case

This section ...

In [1]:
# Loading of the used packages
import numpy as np # basic mathematical and array functions
import os # Functions related to the operating system
import matplotlib.pyplot as plt # Plotting library

import sharpy.sharpy_main
import sharpy.utils.plotutils as pu # Plotting utilities
from sharpy.utils.constants import deg2rad # Constant to conver degrees to radians
import sharpy.utils.generate_cases as gc

The "generate_cases" module of SHARPy tries to facilitate the creation of simple cases using common parameters as inputs and hidding the tedious work. Once the reader has understood the big picture on how to generate a case through this notebook, we recommend to naviate inside the functions to further understand SHARPy and to run the rest of the example notebooks that have a higher complexity.

### Define the case parameters

In [2]:
# Geometry
chord = 1. 
aspect_ratio = 16.
wake_length = 50

# Discretization
num_node = 21
num_chord_panels = 4
num_points_camber = 200

aoa_ini_deg = 2.

# Operation
WSP = 25
air_density = 0.0899

# Time discretization
end_time = 10.0
dt = chord/num_chord_panels/WSP # Always keep one timestep per panel

# Stiffness
mass_per_unit_length = 0.75
mass_iner_x = 0.1
mass_iner_y = 0.05
mass_iner_z = 0.05
pos_cg_B = np.zeros((3))
EA = 1e7
GAy = 1e6
GAz = 1e6
GJ = 1e4
EIy = 2e4 # Flapwise
EIz = 5e6 # Edgewise

In [3]:
%config InlineBackend.figure_format = 'svg'
from IPython.display import Image
url = 'https://raw.githubusercontent.com/ImperialCollegeLondon/sharpy/dev_doc/docs/source/content/example_notebooks/images/simple_airfoil.png'
Image(url=url, width=800)

### Generate SHARPy required inputs

The following cells will create an AeroelasticInformation class. This class is not directly used by SHARPy, it has been thought as an intermediate step between common engineering inputs and the input information that SHARPy needs. 
It has a lot of functionalities (rotate objects, assembly simple strucures ...) but it does not provide as much functionalities as SHARPy. For that reason, we encourage the reader to dive into the functions and the rest of the example notebooks to figure out the real inputs required by SHARPy.

In [4]:
airfoil = gc.AeroelasticInformation()

In [5]:
airfoil.__dict__.keys()
airfoil.StructuralInformation.__dict__.keys()
print(airfoil.StructuralInformation.connectivities)

list_of_methdos = gc.list_methods(airfoil.StructuralInformation)


None
assembly_structures
check_StructuralInformation
compute_basic_num_elem
compute_basic_num_node
copy
create_frame_of_reference_delta
create_mass_db_from_vector
create_simple_connectivities
create_stiff_db_from_vector
generate_1to1_from_vectors
generate_fem_file
generate_full_structure
generate_uniform_beam
generate_uniform_sym_beam
rotate_around_origin
set_to_zero


In [6]:
airfoil.StructuralInformation.num_node = num_node
airfoil.StructuralInformation.num_node_elem = 3
airfoil.StructuralInformation.compute_basic_num_elem()

In [7]:
# Generate an array with the location of the nodes
node_r = np.zeros((num_node,3))
node_r[:,1] = np.linspace(0, chord*aspect_ratio, num_node)

In [8]:
airfoil.StructuralInformation.generate_uniform_beam(node_r,
                    mass_per_unit_length,
                    mass_iner_x,
                    mass_iner_y,
                    mass_iner_z,
                    pos_cg_B,
                    EA,
                    GAy,
                    GAz,
                    GJ,
                    EIy,
                    EIz,
                    num_node_elem = airfoil.StructuralInformation.num_node_elem,
                    y_BFoR = 'x_AFoR',
                    num_lumped_mass=0)

airfoil.StructuralInformation.structural_twist = np.zeros((airfoil.StructuralInformation.num_elem, airfoil.StructuralInformation.num_node_elem),)
#airfoil.StructuralInformation.boundary_conditions = np.zeros((airfoil.StructuralInformation.num_node), dtype = int)
airfoil.StructuralInformation.boundary_conditions[0] = 1
airfoil.StructuralInformation.boundary_conditions[-1] = -1

print(airfoil.StructuralInformation.connectivities)

[[ 0  2  1]
 [ 2  4  3]
 [ 4  6  5]
 [ 6  8  7]
 [ 8 10  9]
 [10 12 11]
 [12 14 13]
 [14 16 15]
 [16 18 17]
 [18 20 19]]


In [9]:
# Define the coordinates of the camber line of the airfoils used
# 
airfoil_camber = np.zeros((1, num_points_camber, 2))
airfoil_camber[0, :, 0] = np.linspace(0, 1, num_points_camber)

# Compute the number of panels in the wake (streamwise direction) based on the previous paramete
mstar = int(wake_length*chord/dt)

In [10]:
# Generate blade aerodynamics
airfoil.AerodynamicInformation.create_one_uniform_aerodynamics(airfoil.StructuralInformation,
                                 chord = chord,
                                 twist = 0.,
                                 sweep = 0.,
                                 num_chord_panels = num_chord_panels,
                                 m_distribution = 'uniform',
                                 elastic_axis = 0.5,
                                 num_points_camber = num_points_camber,
                                 airfoil = airfoil_camber)

let's run a first simulation with SHARPy. It will perform no computation, it will just load the data so we can plot the system we have just created.

In [11]:
# Generate basic information about the case
case = ('aoa%.02f' % aoa_ini_deg).replace(".","p")
route = './'

In [41]:
pip install "notebook>=5.3" "ipywidgets>=7.2"

Note: you may need to restart the kernel to use updated packages.


In [39]:
# Define the simulation
SimInfo = gc.SimulationInformation()
SimInfo.set_default_values()

SimInfo.solvers['SHARPy']['flow'] = ['BeamLoader',
                        'AerogridLoader']

SimInfo.solvers['SHARPy']['case'] = case
SimInfo.solvers['SHARPy']['route'] = route
SimInfo.solvers['SHARPy']['write_screen'] = 'off'

SimInfo.solvers['AerogridLoader']['unsteady'] = 'on'
SimInfo.solvers['AerogridLoader']['mstar'] = mstar
SimInfo.solvers['AerogridLoader']['freestream_dir'] = np.array([1.,0.,0.])
SimInfo.solvers['AerogridLoader']['wake_shape_generator'] = 'StraightWake'
SimInfo.solvers['AerogridLoader']['wake_shape_generator_input'] = {'u_inf': WSP,
                                                                   'u_inf_direction': np.array([np.cos(aoa_ini_deg*deg2rad), 0., np.sin(aoa_ini_deg*deg2rad)]),
                                                                   'dt': dt}

gc.clean_test_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
airfoil.generate_h5_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
SimInfo.generate_solver_file()

sharpy_output = sharpy.sharpy_main.main(['', SimInfo.solvers['SHARPy']['route'] + SimInfo.solvers['SHARPy']['case'] + '.sharpy'])



In [37]:
plot_timestep(sharpy_output, tstep=-1, minus_mstar=(mstar - 6), plotly=True)

In [38]:
SimInfo.solvers['SHARPy']

{'flow': ['BeamLoader', 'AerogridLoader'],
 'case': 'aoa2p00',
 'route': './',
 'write_screen': 'on',
 'write_log': 'off',
 'log_folder': './output',
 'log_file': 'log'}

In [5]:
# Compute the number of time steps needed based on the previous parameters
time_steps = int(end_time/dt)

# Generate basic information about the case
case = ('aoa%.02f' % aoa_ini_deg).replace(".","p")
route = os.path.dirname(os.path.realpath(__file__)) + '/'

# Define the simulation
SimInfo = gc.SimulationInformation()
SimInfo.set_default_values()

SimInfo.solvers['SHARPy']['flow'] = ['BeamLoader',
                        'AerogridLoader',
                        'StaticCoupled',
                        'AerogridPlot',
                        'BeamPlot',
                        'DynamicCoupled']

SimInfo.solvers['SHARPy']['case'] = case
SimInfo.solvers['SHARPy']['route'] = route



NameError: name 'end_time' is not defined

In [5]:
SimInfo.set_variable_all_dicts('rho', air_density)
# SimInfo.set_variable_all_dicts('velocity_field_generator','SteadyVelocityField')
SimInfo.set_variable_all_dicts('dt', dt)
SimInfo.define_num_steps(time_steps)

NameError: name 'air_density' is not defined

In [6]:
SimInfo.solvers['AerogridLoader']['unsteady'] = 'on'
SimInfo.solvers['AerogridLoader']['mstar'] = mstar
SimInfo.solvers['AerogridLoader']['freestream_dir'] = np.array([1.,0.,0.])

NameError: name 'mstar' is not defined

In [7]:
SimInfo.solvers['StaticUvlm']['horseshoe'] = True
SimInfo.solvers['StaticUvlm']['num_cores'] = 8
SimInfo.solvers['StaticUvlm']['n_rollup'] = 0
SimInfo.solvers['StaticUvlm']['rollup_dt'] = dt
SimInfo.solvers['StaticUvlm']['velocity_field_generator'] = 'SteadyVelocityField'
SimInfo.solvers['StaticUvlm']['velocity_field_input'] = {'u_inf' : WSP,
                                                         'u_inf_direction' : np.array([np.cos(aoa_ini_deg*deg2rad), 0., np.sin(aoa_ini_deg*deg2rad)])}


NameError: name 'dt' is not defined

In [8]:
SimInfo.solvers['StaticCoupled']['structural_solver'] = 'NonLinearStatic'
SimInfo.solvers['StaticCoupled']['structural_solver_settings'] = SimInfo.solvers['NonLinearStatic']
SimInfo.solvers['StaticCoupled']['aero_solver'] = 'StaticUvlm'
SimInfo.solvers['StaticCoupled']['aero_solver_settings'] = SimInfo.solvers['StaticUvlm']
SimInfo.solvers['StaticCoupled']['max_iter'] = 100
SimInfo.solvers['StaticCoupled']['n_load_steps'] = 0
SimInfo.solvers['StaticCoupled']['tolerance'] = 1e-08
SimInfo.solvers['StaticCoupled']['relaxation_factor'] = 0.0

SimInfo.solvers['StepUvlm']['convection_scheme'] = 0
SimInfo.solvers['StepUvlm']['num_cores'] = 8
SimInfo.solvers['StepUvlm']['velocity_field_input'] = {'u_inf' : WSP,
                                                       'u_inf_direction' : np.array([1., 0., 0.])}

SimInfo.solvers['WriteVariablesTime']['structure_variables'] = ['pos', 'psi']
SimInfo.solvers['WriteVariablesTime']['structure_nodes'] = list(range(0, num_node))

SimInfo.solvers['NonLinearDynamicPrescribedStep']['max_iterations'] = 100
SimInfo.solvers['NonLinearDynamicPrescribedStep']['num_load_steps'] = 1
SimInfo.solvers['NonLinearDynamicPrescribedStep']['delta_curved'] = 0.01
SimInfo.solvers['NonLinearDynamicPrescribedStep']['min_delta'] = 1e-05
SimInfo.solvers['NonLinearDynamicPrescribedStep']['newmark_damp'] = 0.
SimInfo.solvers['NonLinearDynamicPrescribedStep']['gravity_on'] = False
SimInfo.solvers['NonLinearDynamicPrescribedStep']['gravity'] = 9.81
SimInfo.solvers['NonLinearDynamicPrescribedStep']['relaxation_factor'] = 0.

SimInfo.solvers['DynamicCoupled']['structural_solver'] = 'NonLinearDynamicPrescribedStep'
SimInfo.solvers['DynamicCoupled']['structural_solver_settings'] = SimInfo.solvers['NonLinearDynamicPrescribedStep']
SimInfo.solvers['DynamicCoupled']['aero_solver'] = 'StepUvlm'
SimInfo.solvers['DynamicCoupled']['aero_solver_settings'] = SimInfo.solvers['StepUvlm']
SimInfo.solvers['DynamicCoupled']['postprocessors'] = ['BeamPlot', 'AerogridPlot', 'Cleanup', 'WriteVariablesTime']
SimInfo.solvers['DynamicCoupled']['postprocessors_settings'] = {'BeamPlot': SimInfo.solvers['BeamPlot'],
                                                             'AerogridPlot': SimInfo.solvers['AerogridPlot'],
                                                             'WriteVariablesTime': SimInfo.solvers['WriteVariablesTime'],
                                                             'Cleanup': SimInfo.solvers['Cleanup']}
SimInfo.solvers['DynamicCoupled']['minimum_steps'] = 0
SimInfo.solvers['DynamicCoupled']['include_unsteady_force_contribution'] = True
SimInfo.solvers['DynamicCoupled']['relaxation_factor'] = 0.
SimInfo.solvers['DynamicCoupled']['final_relaxation_factor'] = 0.
SimInfo.solvers['DynamicCoupled']['dynamic_relaxation'] = False
SimInfo.solvers['DynamicCoupled']['relaxation_steps'] = 0
SimInfo.solvers['DynamicCoupled']['fsi_tolerance'] = 1e-8
SimInfo.solvers['DynamicCoupled']['fsi_substeps'] = 70
SimInfo.solvers['DynamicCoupled']['structural_substeps'] = 0
SimInfo.solvers['DynamicCoupled']['steps_without_unsteady_force'] = 0
SimInfo.solvers['DynamicCoupled']['pseudosteps_ramp_unsteady_force'] = 0


NameError: name 'WSP' is not defined

In [9]:
SimInfo.with_forced_vel = True
SimInfo.for_vel = np.zeros((time_steps,6), dtype=float)
SimInfo.for_acc = np.zeros((time_steps,6), dtype=float)
SimInfo.with_dynamic_forces = True
SimInfo.dynamic_forces = np.zeros((time_steps,airfoil.StructuralInformation.num_node,6), dtype=float)

NameError: name 'time_steps' is not defined

In [10]:
gc.clean_test_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
airfoil.generate_h5_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
SimInfo.generate_solver_file()
SimInfo.generate_dyn_file(time_steps)

NameError: name 'airfoil' is not defined

In [11]:
sharpy_output = sharpy.sharpy_main.main(['', SimInfo.solvers['SHARPy']['route'] + SimInfo.solvers['SHARPy']['case'] + '.sharpy'])

NameError: name 'sharpy' is not defined

In [ ]:
# Generate a simple plot of the timestep 

In [ ]:
pu.plot_timestep(data, tstep=-1)

fig, list_plots = plt.subplots(1, 2, figsize=(12, 3))

list_plots[0].grid()
list_plots[0].set_xlabel("r/R [-]")
list_plots[0].set_ylabel("CN/d(r/R) [-]")
list_plots[0].plot(r[0]/R, CN_drR[0], '-', label='SHARPy')
list_plots[0].plot(of_rR, of_cNdrR, '-', label='OpenFAST')
list_plots[0].legend()

list_plots[1].grid()
list_plots[1].set_xlabel("r/R [-]")
list_plots[1].set_ylabel("CT/d(r/R) [-]")
list_plots[1].plot(r[0]/R, CTan_drR[0], '-', label='SHARPy')
list_plots[1].plot(of_rR, of_cTdrR, '-', label='OpenFAST')
list_plots[1].legend()

plt.show()